In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.9/693.9 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.optim import Adam
from collections import OrderedDict
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from torchvision import datasets, transforms, models
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset, random_split
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

### Define Custom Model Class

In [11]:
from IPython.core.display import set_matplotlib_formats
class ConvNet(pl.LightningModule):
    def __init__(self, pretrained_model_name, pretrained_model_path, num_classes, resizing_factor, base_lr, batch_size, train_path, test_path):
        super(ConvNet, self).__init__()

        self.pretrained_model_name = pretrained_model_name
        self.pretrained_model_path = pretrained_model_path
        self.num_classes = num_classes
        self.resizing_factor = resizing_factor
        self.lr = base_lr
        self.batch_size = batch_size
        self.train_path = train_path
        self.test_path = test_path
        self.in_feat = None
        self.model = None

        # dict to store training progress per epoch
        self.history = {'train_loss': [],
               'val_loss': [],
               'train_acc':[],
               'val_acc':[]
               }

        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.test_step_outputs = []

        # transfer learning parameters
        self.classifiers_n = -1
        self.features_n = -1

        # check for GPU availability
        use_gpu = torch.cuda.is_available()

        # load model architectures without weight
        if use_gpu:
            self.model = getattr(models, self.pretrained_model_name)().cuda()
        else:
            self.model = getattr(models, self.pretrained_model_name)()

        # load pre-trained weights
        if use_gpu:
            self.model.load_state_dict(torch.load(self.pretrained_model_path))
        else:
            self.model.load_state_dict(torch.load(self.pretrained_model_path, map_location=torch.device('cpu')))

        # get input dimension of the fc layer to be replaced and index of the last fc layer
        self.in_feat = self.model.classifier[-1].in_features
        fc_idx = len(self.model.classifier) - 1

        custom_fc = nn.Sequential(nn.Linear(self.in_feat, 512),
                    nn.ReLU(),
                    nn.Dropout(0.5),
                    nn.Linear(512, self.num_classes),
                    nn.ReLU(),
                    nn.Dropout(0.5),
                    nn.LogSoftmax(dim=1))

        # add custom fc layers to model
        self.model.classifier[fc_idx] = custom_fc

    def forward(self, x):
        x = self.model(x)
        return x

    # freezes all layers in the model
    def freeze_all_layers(self):
        for param in self.model.parameters():
            param.requires_grad = False

    # unfreeze last 'n' fully connected layers
    def unfreeze_last_n_fc_layers(self, n):

        # if n == -1 don't unfreeze any layers
        if n == -1:
            return 0

        n = n*2 # since weights and bias are included as separate
        total_layers = len(list(self.model.classifier.parameters()))

        # invalid n
        if n > total_layers:
            print(f"Warning: There are only {total_layers} layers in the model. Cannot unfreeze {n} layers.")

        # if n == 0 unfreeze all layers
        elif n == 0:
            for param in self.model.classifier.parameters():
                param.requires_grad = True
        else:
            for i, param in enumerate(self.model.classifier.parameters()):
                if i >= (total_layers - n):
                    param.requires_grad = True
                else:
                    param.requires_grad = False


    # unfreeze last 'n' fully connected layers
    def unfreeze_last_n_conv_layers(self, n):

        # if n == -1 don't unfreeze any layers
        if n == -1:
            return 0

        n = n*2 # since weights and bias are included as separate
        total_layers = len(list(self.model.features.parameters()))

        # invalid n
        if n > total_layers:
            print(f"Warning: There are only {total_layers} layers in the model. Cannot unfreeze {n} layers.")
        # if n == 0 unfreeze all layers
        elif n == 0:
            for param in self.model.features.parameters():
                param.requires_grad = True
        else:
            for i, param in enumerate(self.model.features.parameters()):
                if i >= total_layers - n:
                    param.requires_grad = True
                else:
                    pass

    # set parameters for transfer learning
    def set_transfer_learning_params(self, unfreeze_n_fc, unfreeze_n_conv):
        self.classifier_n = unfreeze_n_fc
        self.features_n = unfreeze_n_conv
        self.freeze_all_layers()
        self.unfreeze_last_n_fc_layers(unfreeze_n_fc)
        self.unfreeze_last_n_conv_layers(unfreeze_n_conv)

    def get_optimizer_params_list(self):
        # list of dictionaries to store parameter values
        params_list = []

        # multiplying factor
        f_fc = 10
        f_conv = 2

        if self.classifier_n != -1:
            if self.classifier_n == 0:
                named_params = list(name for name, _ in self.model.classifier.named_parameters())
                layer_indices = list(OrderedDict.fromkeys([int(name.split('.')[0]) for name in named_params]))
            else:
                # get indices of the last 'n' layers in the model
                named_params = list(name for name, _ in self.model.classifier.named_parameters())
                layer_indices = list(OrderedDict.fromkeys([int(name.split('.')[0]) for name in named_params[-self.classifier_n*2:]]))
            for i, index_val in enumerate(layer_indices):
                #params_list.append({'params':self.model.classifier[index_val].parameters(), 'lr': self.lr*f_fc})
                params_list.append({'params':self.model.classifier[index_val].parameters(), 'lr': 0.001})

        if self.features_n != -1:
            if self.features_n == 0:
                named_params = list(name for name, _ in self.model.features.named_parameters())
                layer_indices = list(OrderedDict.fromkeys([int(name.split('.')[0]) for name in named_params]))
            else:
                # get indices of the last 'n' layers in the model
                named_params = list(name for name, _ in self.model.features.named_parameters())
                layer_indices = list(OrderedDict.fromkeys([int(name.split('.')[0]) for name in named_params[-self.features_n*2:]]))
            for i, index_val in enumerate(layer_indices):
                # params_list.append({'params':self.model.features[index_val].parameters(), 'lr': self.lr*(f_conv*(i+1))})
                params_list.append({'params':self.model.features[index_val].parameters(), 'lr': 0.001})

        if self.classifier_n == self.features_n == -1:
            return self.model.parameters()

        return params_list

    def configure_optimizers(self):
        params_list = self.get_optimizer_params_list()
        optimizer = Adam(params_list, lr = self.lr)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        logps = self(x)
        loss = F.nll_loss(logps, y)
        y_pred = torch.argmax(torch.exp(logps), 1)
        acc = (y_pred == y).sum().item()/len(y)
        self.log('train_loss', loss.item(), on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.training_step_outputs.append((loss.item(), acc))
        return loss

    def on_train_epoch_end(self):
        num_items = len(self.training_step_outputs)
        cum_loss = 0
        cum_acc = 0
        for loss, acc in self.training_step_outputs:
            cum_loss += loss
            cum_acc += acc

        self.history['train_loss'].append(cum_loss/num_items)
        self.history['train_acc'].append(cum_acc/num_items)
        self.training_step_outputs.clear()

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logps = self(x)
        loss = F.nll_loss(logps, y)
        y_pred = torch.argmax(torch.exp(logps), 1)
        acc = (y_pred == y).sum().item()/len(y)
        self.log('val_loss', loss.item(), on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.validation_step_outputs.append((loss.item(), acc))
        return loss

    def on_validation_epoch_end(self):
        num_items = len(self.validation_step_outputs)
        cum_loss = 0
        cum_acc = 0
        for loss, acc in self.validation_step_outputs:
            cum_loss += loss
            cum_acc += acc

        self.history['val_loss'].append(cum_loss/num_items)
        self.history['val_acc'].append(cum_acc/num_items)
        self.validation_step_outputs.clear()

    def test_step(self, batch, batch_idx):
        x, y = batch
        logps = self(x)
        loss = F.nll_loss(logps, y)
        y_pred = torch.argmax(torch.exp(logps), 1)
        acc = (y_pred == y).sum().item()/len(y)
        self.log('test_loss', loss.item(), on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('test_acc', acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.test_step_outputs.append((loss.item(), acc))
        return loss

    # create dataset objects
    def setup(self, stage=None):

        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]

        # define transformers
        train_transform = transforms.Compose([
                transforms.Resize(self.resizing_factor),
                transforms.RandomHorizontalFlip(0.5),
                transforms.RandomRotation(15),
                transforms.RandomAffine(degrees = 10,
                                        translate = (0.2, 0.2), shear = 10),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)])

        test_transform = transforms.Compose([transforms.Resize(self.resizing_factor),
                                            transforms.ToTensor(),
                                            transforms.Normalize(mean, std)])

        # create datasets
        train = torchvision.datasets.ImageFolder(self.train_path, transform=train_transform)
        total_items = len(train)
        val_size = int(total_items*0.2)
        train_size = total_items - val_size
        self.train_dataset, self.val_dataset = random_split(train, [train_size, val_size])
        self.test_dataset = torchvision.datasets.ImageFolder(self.test_path, transform=test_transform)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, self.batch_size, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, self.batch_size, shuffle=False)

    def get_history(self):
        return self.history


### Helper Functions

In [12]:
# plot history
def plot_history(history):
    train_loss = history['train_loss']
    val_loss = history['val_loss']
    train_acc = history['train_acc']
    val_acc = history['val_acc']

    # Plot train_loss vs. val_loss
    plt.figure(figsize=(10, 8))
    plt.subplot(2, 1, 1)
    plt.plot(train_loss, label='Train Loss', color='blue')
    plt.plot(val_loss, label='Validation Loss', color='red')
    plt.title('Training Vs Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot train_acc vs. val_acc
    plt.subplot(2, 1, 2)
    plt.plot(train_acc, label='Train Accuracy', color='blue')
    plt.plot(val_acc, label='Validation Accuracy', color='red')
    plt.title('Training Vs Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Adjust spacing between subplots
    plt.tight_layout()

### Transfer Learning Modules

In [13]:
def train_custom_fc_layers(trainer, model):
    # freeze all layers except the last two fc layers
    unfreeze_n_fc = 2
    unfreeze_n_conv = -1
    model.set_transfer_learning_params(unfreeze_n_fc, unfreeze_n_conv)

    # train model
    trainer.fit(model)

    # get training history
    history = model.get_history()

    # plot history
    plot_history(history)

    return history

def train_entire_fc_block(trainer, model):
    # freeze all layers except the fc block
    unfreeze_n_fc = 0
    unfreeze_n_conv = -1
    model.set_transfer_learning_params(unfreeze_n_fc, unfreeze_n_conv)

    # train model
    trainer.fit(model)

    # get training history
    history = model.get_history()

    # plot history
    plot_history(history)

    return history

def train_conv_layers(trainer, model):
    # freeze all layers except the last two conv layers
    unfreeze_n_fc = -1
    unfreeze_n_conv = 2
    model.set_transfer_learning_params(unfreeze_n_fc, unfreeze_n_conv)

    # train model
    trainer.fit(model)

    # get training history
    history = model.get_history()

    # plot history
    plot_history(history)

    return history

def fine_tune_model(trainer, model):
    # freeze all layers except the last two conv layers and the fc block
    unfreeze_n_fc = 0
    unfreeze_n_conv = 2
    model.set_transfer_learning_params(unfreeze_n_fc, unfreeze_n_conv)

    # train model
    trainer.fit(model)

    # get training history
    history = model.get_history()

    # plot history
    plot_history(history)

    return history

In [14]:
# define variables
pretrained_model_name = 'vgg16'
pretrained_model_path = '/content/drive/MyDrive/Colab Notebooks/pretrained_models/vgg16.pth'
num_classes = 10
resizing_factor = (224, 224)
base_lr = 0.001
epochs = 10
batch_size = 32
train_path = '/content/drive/MyDrive/Colab Notebooks/Data/CUB_200_2011/train_test_cropped/train'
test_path = '/content/drive/MyDrive/Colab Notebooks/Data/CUB_200_2011/train_test_cropped/test'
log_dir = '/content/drive/MyDrive/Colab Notebooks/logs/lightning_logs'

In [15]:
# logger
logger = TensorBoardLogger(log_dir)

# load trainer
trainer = pl.Trainer(limit_train_batches=20, limit_test_batches=10, limit_val_batches=10,
                         max_epochs = epochs, logger = logger)
# create model
model = ConvNet(pretrained_model_name, pretrained_model_path, num_classes, resizing_factor,
                base_lr, batch_size, train_path, test_path)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [16]:
history = train_custom_fc_layers(trainer, model)

INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type | Params
-------------------------------
0 | model | VGG  | 136 M 
-------------------------------
2.1 M     Trainable params
134 M     Non-trainable params
136 M     Total params
545.453   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

IndexError: ignored

In [ ]:
history

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir ./lightning_logs